<h1 align='center'>predictions</h1>

input weights and images<br>
output x and y co-ordinate predictions

In [ ]:
import shutil, stat
import torch
import os
import glob
import cv2 as cv
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import json
import numpy as np
from PIL import Image
%matplotlib inline

# METHOD 2 location of cell detection model weights
cell_model_path = r"C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\methods\method_2\weights\\"

# METHOD 3 location of cell and spot detection model weights
tab_model_path = r"C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\methods\method_3\weights\\"
spot_model_path = r"C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\methods\method_3\weights\\"

# METHOD 4 location of tab detection model config
model_path = r"C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\methods\method_4\weights\\"


module_base = r"C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\evaluation\\"
# MODULE 1 location of test_image and output
module_1_base = module_base + "module_1\\"
module_1_path = module_1_base + 'test_image.jpg'

# MODULE 2 location of test_image
module_2_base = module_base + "module_2\\"
module_2_path = module_2_base + 'test_image.jpg'

# MODULE 3 location of test_image
module_3_base = module_base + "module_3\\"
module_3_path = module_3_base + 'test_image.jpg'

# MODULE 4 location of test_image
module_4_base = module_base + "module_4\\"
module_4_path = module_4_base + 'test_image.jpg'


yolov5_path = r"C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\methods\working\yolov5"

In [ ]:
def reset_yolov5():
    %cd "C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\methods\working"

    def on_rm_error(func, path, exc_info):
        # path contains the path of the file that couldn't be removed
        # let's just assume that it's read-only and unlink it.
        os.chmod(path, stat.S_IWRITE )
        os.unlink(path)

    shutil.rmtree('yolov5', onerror = on_rm_error)

    !git clone https://github.com/ultralytics/yolov5

    %cd yolov5
    %pip install -qr requirements.txt 

    %cd ..
    from yolov5 import utils, val
    display = utils.notebook_init() #check

    %cd ..

In [ ]:
def find_image_path(module):
    # location of test image
    if module == 1:
        image_path = module_1_path
    elif module == 2:
        image_path = module_2_path
    elif module == 3:
        image_path = module_3_path
    elif module == 4:
        image_path = module_4_path
        
    return image_path

In [ ]:
def find_module_path(module):
    
    if module == 1:
        module_path = 'module_1'
    elif module == 2:
        module_path = 'module_2'
    elif module == 3:
        module_path = 'module_3'
    elif module == 4:
        module_path = 'module_4'
    
    return module_path

In [ ]:
def cv_image_display(img, x_dim, y_dim):
    
    # resize to fit window
    y_sf = 0.3
    x_sf = 0.3
    y_dim_rs = int(y_dim*y_sf)
    x_dim_rs = int(x_dim*x_sf)
    img = cv.resize(img, (x_dim_rs, y_dim_rs)) 
    
    # display image
    cv.imshow('img', img)
    cv.waitKey(0)

#### method 1

In [ ]:
class HoughParams:
    
    def __init__(self, dp, min_dist, param_1, param_2, min_radius, max_radius):
        self.dp = dp
        self.min_dist = min_dist
        self.param_1 = param_1
        self.param_2 = param_2
        self.min_radius = min_radius
        self.max_radius = max_radius
    
# module_1_params = HoughParams(dp = 1, min_dist = 100, param_1 = 180, param_2 = 19, min_radius = 115, max_radius = 130)
# module_2_params = HoughParams(dp = 1, min_dist = 100, param_1 = 115, param_2 = 22, min_radius = 50, max_radius = 60)
# module_3_params = HoughParams(dp = 1, min_dist = 100, param_1 = 85, param_2 = 30, min_radius = 105, max_radius = 120)
# module_4_params = HoughParams(dp = 1, min_dist = 100, param_1 = 30, param_2 = 30, min_radius = 123, max_radius = 135)

module_1_params = HoughParams(dp = 1, min_dist = 100, param_1 = 180, param_2 = 19, min_radius = 115, max_radius = 130)
module_2_params = HoughParams(dp = 1, min_dist = 100, param_1 = 180, param_2 = 19, min_radius = 50, max_radius = 60)
module_3_params = HoughParams(dp = 1, min_dist = 100, param_1 = 180, param_2 = 19, min_radius = 105, max_radius = 120)
module_4_params = HoughParams(dp = 1, min_dist = 100, param_1 = 180, param_2 = 19, min_radius = 123, max_radius = 135)

In [ ]:
def hough(module, dp, min_dist, param_1, param_2, min_radius, max_radius):
    
    image_path = find_image_path(module)
    output_path = '\\'.join(image_path.split('\\')[0:-1]) + '\\method_1_predicted\\'

    img = cv.imread(image_path)
    y_dim = img.shape[0]
    x_dim = img.shape[1]

    gray = np.copy(cv.cvtColor(img, cv.COLOR_BGR2GRAY))
    img = cv.medianBlur(gray,5)
    cimg = np.copy(cv.cvtColor(img,cv.COLOR_GRAY2BGR))


    circles = cv.HoughCircles(img, cv.HOUGH_GRADIENT, dp, min_dist, 
                              param1=param_1, param2=param_2, 
                              minRadius=min_radius, maxRadius=max_radius)
    
    circles = np.uint16(np.around(circles))

    for i in circles[0,:]:
        # draw the outer circle
        cv.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
        # draw the center of the circle
        cv.circle(cimg,(i[0],i[1]),2,(0,0,255),3)

    cv_image_display(cimg, x_dim, y_dim)

    xs = []
    ys = []

    for circle in circles[0]:
        x = int(circle[0])
        y = int(circle[1])

        xs.append(x)
        ys.append(y)
        
    return xs, ys

In [ ]:
def method_1(module):
    
    image_path = find_image_path(module)
    output_path = '\\'.join(image_path.split('\\')[0:-1]) + '\\method_1_predicted\\'

    if module == 1:
        params = module_1_params
    elif module == 2:
        params = module_2_params
    elif module == 3: 
        params = module_3_params
    elif module == 4:
        params = module_4_params

    xs, ys = hough(module, params.dp, params.min_dist,
                              params.param_1, params.param_2,
                              params.min_radius, params.max_radius)
        
    im = mpimg.imread(image_path)
    fig = plt.imshow(im)
    plt.scatter(xs,ys, c='r', marker="x", alpha=0.7, linewidth=0.5)
    plt.show
    plt.savefig(output_path + 'test_image' + '.png', dpi=500)

    with open(output_path + 'test_image' + '-x.json', "w") as fp:
        json.dump(xs, fp)

    with open(output_path + 'test_image' + '-y.json', "w") as fp:
        json.dump(ys, fp)

In [ ]:
for module in range(1, 5):
    method_1(module)

---

#### method 2

In [ ]:
reset_yolov5()

In [ ]:
def method_2(module):
    
#     reset_yolov5()
    
    image_path = find_image_path(module)
    output_path = '\\'.join(image_path.split('\\')[0:-1]) + '\\method_2_predicted\\'
    %cd {yolov5_path}
    
    # infer on module image, extracting all bbox image and label data
    !python detect.py \
    --weights "C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\methods\method_2\weights\cells\temp.pt" \
    --img 640 \
    --conf 0.12 \
    --source "C:\\Users\\felix\\OneDrive - University of Bristol\\Year 4\\1. Design Project 4\\Project\\weld-spot-prediction\\evaluation\\module_4\\test_image.jpg" \
    --save-crop \
    --save-txt \
    --project detect-cells
    
    # find latest directory made in /detect-tabs
    directory = './detect-cells'
    tab_results = max(glob.glob(os.path.join(directory, '*/')), key=os.path.getmtime)

    tabs_keys = ['tab_image_x_center', 'tab_image_y_center', 'tab_image_width', 'tab_image_height']
    df = pd.DataFrame(columns=[tabs_keys])
    cwd = os.getcwd() + '\\' + tab_results.split('.')[1][1:]

    annot_path = cwd + '/labels/' 
    for file in glob.glob(annot_path + '/*.txt'):
        with open(file, 'r') as f:
            lines = f.readlines()

    annots = [line.replace("\n","").split(' ') for line in lines]
    annots = [annot[1:] for annot in annots]
    annots = [[float(num) for num in annot] for annot in annots]

    for i, annot in enumerate(annots):
        for j, k in enumerate(tabs_keys):
            df.loc[i, k] = annot[j]

    source_img_height, source_img_width = 3024, 4032

    xs, ys = [], [] 

    for index, row in df.iterrows():
        xs.append(row.tab_image_x_center * source_img_width)
        ys.append(row.tab_image_y_center * source_img_height)

#     im = mpimg.imread(image_path)
#     fig = plt.imshow(im)
#     plt.scatter(xs,ys, c='r', marker="x", alpha=0.7, linewidth=0.5)
#     plt.show
#     plt.savefig(output_path + 'test_image' + '.png', dpi=500)

    with open(output_path + 'test_image' + '-x.json', "w") as fp:
        json.dump(xs, fp)

    with open(output_path + 'test_image' + '-y.json', "w") as fp:
        json.dump(ys, fp)

In [ ]:
%%time
method_2(4)

In [ ]:
%cd {yolov5_path}

# infer on module image, extracting all bbox image and label data
!python detect.py \
--weights "C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\methods\method_2\weights\cells\modules_1-3_best_ever.pt" \
--img 640 \
--conf 0.72 \
--source "C:\\Users\\felix\\OneDrive - University of Bristol\\Year 4\\1. Design Project 4\\Project\\weld-spot-prediction\\evaluation\\tests\\" \
--save-txt \
--project detect-cells

---

#### method 3
- cell detection
- spot detection (where necessary, i.e. for module 1)

In [ ]:
def method_3(module):
    
    reset_yolov5()
    image_path = find_image_path(module)
    output_path = '\\'.join(image_path.split('\\')[0:-1]) + '\\method_3_predicted\\'
    %cd {yolov5_path}

    # infer on module image, extracting all bbox image and label data
    !python detect.py \
    --weights "C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\methods\method_3\weights\cells\best_3.2.pt" \
    --img 1024 \
    --conf 0.5 \
    --source "C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\evaluation\module_1\\test_image.jpg" \
    --save-crop \
    --save-txt \
    --project detect-tabs

    # find latest directory made in /detect-tabs
    directory = './detect-tabs'
    tab_results = max(glob.glob(os.path.join(directory, '*/')), key=os.path.getmtime)

    images = tab_results + 'crops/tabs/'
    # infer on cropped images
    !python detect.py \
    --weights "C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\methods\method_3\weights\spots\best_3.1.pt" \
    --img 128 \
    --conf 0.5 \
    --source {images} \
    --save-txt \
    --project detect-spots \
    --max-det 2  
    # ISSUE: setting this max-det is going to be an issue when generalising to new weld types 

    # find latest directory made in /detect-spots
    directory = './detect-spots'
    spot_results = max(glob.glob(os.path.join(directory, '*/')), key=os.path.getmtime)


    # create dataframe with image id's, dimensions, and annotation data for all boxes detected from an image
    spots_keys = ['spot_annots_x_centers', 'spot_annots_y_centers']
    tabs_keys = ['tab_image_x_center', 'tab_image_y_center', 'tab_image_width', 'tab_image_height']
    df = pd.DataFrame(columns=['tab_image_id'] + tabs_keys + ['tab_image_height (px)', 'tab_image_width (px)'] + spots_keys)

    i = 0
    cwd = os.getcwd() + '\\' + spot_results.split('.')[1][1:]

    for file in glob.glob(cwd+'/*.jpg'):

        entry = {}

        ID = file.split('\\')[-1].split('.')[0]
        im = cv.imread(file)
        dim = im.shape
        h, w = dim[0], dim[1]

        entry['tab_image_id'] = ID
        entry['tab_image_height (px)'] = h
        entry['tab_image_width (px)'] = w

        annot_path = cwd + '/labels/' + ID +'.txt'
        with open(annot_path, 'r') as f:
            lines = f.readlines()
        annots = [line.replace("\n","").split(' ') for line in lines]
        annots = [annot[1:] for annot in annots]
        annots = [[float(num) for num in annot] for annot in annots]

        for j, k in enumerate(spots_keys):
            entry[k] = [annot[j] for annot in annots]

        df.loc[i] = entry

        i += 1
        
        
        
    cwd = os.getcwd() + '\\' + tab_results.split('.')[1][1:]

    annot_path = cwd + '/labels/' 
    for file in glob.glob(annot_path + '/*.txt'):
        with open(file, 'r') as f:
            lines = f.readlines()

    annots = [line.replace("\n","").split(' ') for line in lines]
    annots = [annot[1:] for annot in annots]
    annots = [[float(num) for num in annot] for annot in annots]

    for i, annot in enumerate(annots):
        for j, k in enumerate(tabs_keys):
            df.loc[i, k] = annot[j]
            
            
            
            
    source_img_height, source_img_width = 3024, 4032

    xs, ys = [], [] 

    for index, row in df.iterrows():
        tx = row.tab_image_x_center
        ty = row.tab_image_y_center
        th = row.tab_image_height
        tw = row.tab_image_width

        sx = row.spot_annots_x_centers
        sy = row.spot_annots_y_centers


        for i in range(len(sx)):
            xs.append((tx + (sx[i] - 0.5) * tw) * source_img_width)
            ys.append((ty + (sy[i] - 0.5) * th) * source_img_height)


    im = mpimg.imread(image_path)
    fig = plt.imshow(im)
    plt.scatter(xs,ys, c='r', marker="x", alpha=0.7, linewidth=0.5)
    plt.show
    plt.savefig(output_path + 'test_image' + '.png', dpi=500)

    with open(output_path + 'test_image' + '-x.json', "w") as fp:
        json.dump(xs, fp)

    with open(output_path + 'test_image' + '-y.json', "w") as fp:
        json.dump(ys, fp)

In [ ]:
# method_3(1)

In [ ]:
def method_3_2(module):
    
    reset_yolov5()

    image_path = find_image_path(module)
    output_path = '\\'.join(image_path.split('\\')[0:-1]) + '\\method_3_predicted\\'
    module_path = find_module_path(module)
    %cd {yolov5_path}


    # infer on module image, extracting all bbox image and label data
    !python detect.py \
    --weights "C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\methods\method_3\weights\cells\\{module_path}_best.pt" \
    --img 1024 \
    --conf 0.40 \
    --source "C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\evaluation\\{module_path}\\test_image.jpg" \
    --save-crop \
    --save-txt \
    --project detect-tabs


    # find latest directory made in /detect-tabs
    directory = './detect-tabs'
    tab_results = max(glob.glob(os.path.join(directory, '*/')), key=os.path.getmtime)


    tabs_keys = ['tab_image_x_center', 'tab_image_y_center', 'tab_image_width', 'tab_image_height']
    df = pd.DataFrame(columns=[tabs_keys])
    cwd = os.getcwd() + '\\' + tab_results.split('.')[1][1:]


    annot_path = cwd + '/labels/' 
    for file in glob.glob(annot_path + '/*.txt'):
        with open(file, 'r') as f:
            lines = f.readlines()

    annots = [line.replace("\n","").split(' ') for line in lines]
    annots = [annot[1:] for annot in annots]
    annots = [[float(num) for num in annot] for annot in annots]

    for i, annot in enumerate(annots):
        for j, k in enumerate(tabs_keys):
            df.loc[i, k] = annot[j]


    source_img_height, source_img_width = 3024, 4032

    xs, ys = [], [] 

    for index, row in df.iterrows():
        xs.append(row.tab_image_x_center * source_img_width)
        ys.append(row.tab_image_y_center * source_img_height)

    im = mpimg.imread(image_path)
    fig = plt.imshow(im)
    plt.scatter(xs,ys, c='r', marker="x", alpha=0.7, linewidth=0.5)
    plt.show
    plt.savefig(output_path + 'test_image' + '.png', dpi=500)

    with open(output_path + 'test_image' + '-x.json', "w") as fp:
        json.dump(xs, fp)

    with open(output_path + 'test_image' + '-y.json', "w") as fp:
        json.dump(ys, fp)

---

#### method 4

- single object detector trained to recognise both cells and spots

In [ ]:
def method_4(module):
    
    reset_yolov5()
    image_path = find_image_path(module)
    output_path = '\\'.join(image_path.split('\\')[0:-1]) + '\\method_4_predicted\\'
    
    # infer on module image, extracting all bbox image and label data
    %cd {yolov5_path}
    #     !python detect.py --weights {tab_model_path} --img 1024 --conf 0.5 --source {image_path} --save-crop --save-txt --project 'detect-tabs'
    !python detect.py --weights "C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\methods\method_4\weights\best_4.1.pt" --img 1024 --conf 0.3 --source "C:\Users\felix\OneDrive - University of Bristol\Year 4\1. Design Project 4\Project\weld-spot-prediction\evaluation\module_1\\test_image.jpg" --save-crop --save-txt --project detect-spots
    
    
    %cd {yolov5_path}
    # find latest directory made in /detect-spots
    directory = './detect-spots'
    spot_results = max(glob.glob(os.path.join(directory, '*/')), key=os.path.getmtime)

    %cd {spot_results.split('.')[1][1:]}

    # create dataframe with image id's, dimensions, and annotation data for all boxes detected from an image
    spots_keys = ['spot_annot_x_center', 'spot_annot_y_center']
    df = pd.DataFrame(columns=spots_keys)
    
    i = 0
    %cd {yolov5_path}
    cwd = os.getcwd() + '\\' + spot_results.split('.')[1][1:]
    annot_path = cwd + '/labels/' + 'test_image' +'.txt'

    with open(annot_path, 'r') as f:
        lines = f.readlines()

    annots = [line.replace("\n","").split(' ') for line in lines]

    for i in range(10): # performing this only once doesn't delete all '0' label entries for some unknown reason
        for annot in annots:
            if annot[0] == '0':
                annots.remove(annot)

    annots = [annot[1:] for annot in annots]
    annots = [[float(num) for num in annot] for annot in annots]
    
    for i, annot in enumerate(annots):

        entry = {}
        
        for j, k in enumerate(spots_keys):
            entry[k] = annot[j]
            
        df.loc[i] = entry
        i += 1

        
    source_img_height, source_img_width = 3024, 4032

    %cd {yolov5_path}

    xs, ys = [], [] 

    for index, row in df.iterrows():
        xs.append(row.spot_annot_x_center * source_img_width)
        ys.append(row.spot_annot_y_center * source_img_height)
        

    im = mpimg.imread(image_path)
    fig = plt.imshow(im)
    plt.scatter(xs,ys, c='r', marker="x", alpha=0.7, linewidth=0.5)
    plt.show
    plt.savefig(output_path + 'test_image' + '.png', dpi=500)

    with open(output_path + 'test_image' + '-x.json', "w") as fp:
        json.dump(xs, fp)

    with open(output_path + 'test_image' + '-y.json', "w") as fp:
        json.dump(ys, fp)

In [ ]:
method_4(1)